In [30]:
#
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [31]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Cust-close with CPFP

Generate all the priv/public keys

In [32]:
# Generate example priv/pubkeys
funding_privkey_hex = "1111111111111111111111111111111100000000000000000000000000000000"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

change_privkey_hex = "1111111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

cust_payout_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
cust_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_payout_privkey_hex)).hex()

merch_payout_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_payout_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_payout_privkey_hex)).hex()

merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"
RL = hashlib.sha256(bytes.fromhex(revocation_secret_hex)).digest()
revocation_lock_hex = RL.hex()

merch_cpfp_privkey_hex = "2222222222222222222222222222222277777777777777777777777777777777"
merch_cpfp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_cpfp_privkey_hex)).hex()

cust_cpfp_privkey_hex = "3322222222222222222222222222222277777777777777777777777777777777"
cust_cpfp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_cpfp_privkey_hex)).hex()

# merch_fee_privkey_hex = "2222222222222222222222222222222266666666666666666666666666666666"
# merch_fee_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_fee_privkey_hex)).hex()

In [46]:
# cust priv/pubkeys for executing cpfp
cust1_privkey_hex = "0881111111111111111111111111111100000000000000000000000000000000"
cust1_privkey = bytes.fromhex(merch1_privkey_hex)
cust1_pubkey = privkey_to_pubkey(merch1_privkey)

cust2_privkey_hex = "8881111111111111111111111111111100000000000000000000000000000000"
cust2_privkey = bytes.fromhex(merch2_privkey_hex)
cust2_pubkey = privkey_to_pubkey(merch2_privkey).hex()

cust3_privkey_hex = "7771111111111111111111111111111100000000000000000000000000000000"
cust3_privkey = bytes.fromhex(merch3_privkey_hex)
cust3_pubkey = privkey_to_pubkey(merch3_privkey).hex()

### Start up regtest mode
Delete any history so we are starting from scratch. <br>
Mine 101 blocks so we can spend some btc.

In [34]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
# os.system("bitcoind -regtest -daemon -minrelaytxfee=0")
os.system("bitcoind -regtest -daemon")
time.sleep(2)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [35]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2MuuVDMJqfh2J9iS3DtfUeU1tEfff1SNahi


### Send btc to the escrow funder

In [36]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 3.0")
print(txid_1)

271ea7684b0af1a7c8533bc1b309d66074f410c56e1ccd24320d1317b7f59dfa


In [37]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [38]:
os.system("bitcoin-cli -regtest generate 1");

## Create Funding tx

In [39]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_changev2.py" 
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --input_amount_btc " + "3.0"
                        + " --funding_privkey " + funding_privkey_hex
                        + " --escrow_value_btc " + "2.1"
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_change_value_btc " + "0.89"
                        + " --cust_change_pubkey " + change_pubkey_hex)
print("serialized funding tx:\n" + raw_escrow_tx)

# Broadcast funding tx
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with an 'absurdly' high tx fee

print("\nfunding txid:\n"+escrow_txid)

serialized funding tx:
02000000000101fa9df5b717130d3224cd1c6ec510f47460d609b3c13b53c8a7f10a4b68a71e270100000017160014578dd1183845e18d42f90b1a9f3a464675ad2440ffffffff028058840c00000000220020666c6bfa88ba97c90cb04c7038d56b5854e71a4dd174d79b1260c822a14f791e40084e0500000000160014fc7250a211deddc70ee5a2738de5f07817351cef02473044022042e5fdf9903c359b53d6211b2e9e95f84299bd485f71652a6f0d949c526695c902203abcdfbfcd3030c10e22f445656f2b5c0b0073b8f9c82de93c1f0bf0bfb2ecb30121021d5c5194a62b272f98a7d8321e4b3a03add9d61d13a15a00123888b81850cee200000000

funding txid:
da8db106cd86a2fe4a6aa72e5d98ce60e96eb645632f7cb0958a3be438f63c3e


In [40]:
os.system("bitcoin-cli -regtest generate 1");

#### Decode escrow funding transaction

In [41]:
# raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

## Cust-close from Escrow with CPFP

In [42]:
cust_close_cpfp_tx = subprocess.getoutput("python cust_close_with_cpfp.py"        
                + " --spend_from " + "escrow"
                + " --txid_str " + escrow_txid
                + " --index " + "0"
                + " --input_amount_btc " + "2.1"
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex                                           
                + " --cust_script_value_btc " + "0.98"
                + " --cust_payout_pubkey " + cust_payout_pubkey_hex
                + " --to_self_delay " + "05cf"
                + " --merch_payout_value_btc " + "1"
                + " --merch_payout_pubkey " + merch_payout_pubkey_hex
                + " --revocation_lock " + revocation_lock_hex
                + " --merch_dispute_pubkey " + merch_disp_pubkey_hex
                + " --cust_cpfp_value_btc " + "0.099"
                + " --cust_cpfp_pubkey " + cust_pubkey_hex)
                                          
print("cust close with cpfp tx:\n" + cust_close_cpfp_tx)

cust close with cpfp tx:
020000000001013e3cf638e43b8a95b07c2f6345b66ee960ce985d2ea76a4afea286cd06b18dda0000000000ffffffff04805cd7050000000022002067cb20e705c4eb4363194a74d2f743afc1c9ee3cd741d45e21268b16add04f8b00e1f50500000000160014d4354803d10e77eccfc3bf06c152ae694d05d3810000000000000000436a41f8345a21a55dc665b65c8dcfb49488b8e4f337d5c9bb843603f7222a892ce94103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd768641e00f9700000000001600145d6f6add4b70012131dbb8f0a7b067b70ec6a76f0400483045022100bd93490dedb197c3a736b92cf2c5862ee94fcd589e30e6328cb3a1aca2f1fe7702202fdbb6c019c1230f25bb4d72048bf5486f326376885a034f1ff6d6328706569e0147304402206dee41f5d0f50097498e48438cff68804d1118b941d2fbe0053a95e4322fb78802206d5dcd44bf5fdbab77d2bef5c40b042fbb193d1bf46e29fbfbf578ae688c5e9c0147522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


This tx could be broadcast by itself and it would be a valid tx (cell below). If the fee was too small, we can effectively bump up the fee by creating a 'child' tx that spends from it

In [43]:
cust_close_cpfp_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + cust_close_cpfp_tx)
print(cust_close_cpfp_txid) 

fa8687450943044bad065f0503e87b100eed1e0d6e015454d2b8e7af3f01c83f


In [44]:
os.system("bitcoin-cli -regtest generate 1");

In [45]:
mined_cust_close_cpfp_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_close_cpfp_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + mined_cust_close_cpfp_tx)
print(decoded)

{
  "txid": "fa8687450943044bad065f0503e87b100eed1e0d6e015454d2b8e7af3f01c83f",
  "hash": "7eaf76af8f868e1b9573eaf252f393836054321691b355bc811bc487689981ae",
  "version": 2,
  "size": 453,
  "vsize": 288,
  "weight": 1149,
  "locktime": 0,
  "vin": [
    {
      "txid": "da8db106cd86a2fe4a6aa72e5d98ce60e96eb645632f7cb0958a3be438f63c3e",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100bd93490dedb197c3a736b92cf2c5862ee94fcd589e30e6328cb3a1aca2f1fe7702202fdbb6c019c1230f25bb4d72048bf5486f326376885a034f1ff6d6328706569e01",
        "304402206dee41f5d0f50097498e48438cff68804d1118b941d2fbe0053a95e4322fb78802206d5dcd44bf5fdbab77d2bef5c40b042fbb193d1bf46e29fbfbf578ae688c5e9c01",
        "522102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce902103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value

## Cust-fee input tx

fund another input that the merchant will use to add a large fee to the 'child' transaction.

In [47]:
# address to fund the merchant child tx
cust1_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(cust1_pubkey, testnet = True)
# print("Address: " + cust1_p2sh_p2wpkh_address)

# Fund the merchant child input
txid_2 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + cust1_p2sh_p2wpkh_address + " 1.01")
print("txid:" + txid_2)

# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_2)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == cust1_p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

txid:bd34a95256600ef003ba841b1c5f9a482c4c822658ac95f5112ede7df97bf060
index: 0


In [48]:
os.system("bitcoin-cli -regtest generate 1");

In [49]:
cust1_tx = subprocess.getoutput("python p2nsh_to_p2wpkh.py" 
                        + " --txid " + txid_2
                        + " --index " + str(index)
                        + " --input_amount_btc " + "1.01"
                        + " --input_privkey " + merch1_privkey_hex
                        + " --payout_value_btc " + "1"
                        + " --payout_pubkey " + merch2_pubkey)
# print(cust1_tx)

cust1_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + cust1_tx)
# "true" flag means we are okay with absurdly high tx fee
print("cust fee outpoint:\n" + cust1_txid)

cust fee outpoint:
5102f9d77b313e6838afd8463537f9a3b35f0469fd96392c8c4ee3bd1d9b85e9


## Create child transaction
Input[0]: Child outpoint <br>
Input[1]: Merch p2wpkh outpoint

Output[0]: Merch p2wpkh

In [50]:
cust_child_tx = subprocess.getoutput("python merch_close_child_tx.py" 
                        + " --child_txid_str " + cust_close_cpfp_txid
                        + " --child_index " + "3"
                        + " --child_privkey " + cust_cpfp_privkey_hex
                        + " --child_input_amount_btc " + "0.099"
                        + " --merch_txid_str " + cust1_txid
                        + " --merch_index " + "0"
                        + " --merch_privkey " + cust2_privkey_hex
                        + " --merch_input_amount_btc " + "1"
                        + " --output_value_btc " + "1.008"
                        + " --payout_pubkey " + cust3_pubkey)
print(cust_child_tx)

020000000001023fc8013fafe7b8d25454016e0d1eed0e107be803055f06ad4b044309458786fa0300000000ffffffffe9859b1dbde34e8c2c3996fd69045fb3a3f9373546d8af38683e317bd7f902510000000000ffffffff0100160206000000001600140cbd8eef39d742140b81cf2f7fbade71af58a182024730440220240080e953a67d68c6aaf7f7709d7ee0500d6a8b35795d27c9c7a363e24e09cb02202f8d4ae266a1f61776a6df7b4c512c308e4d6ab183547d77c463ca08a54fce440121029ee20dd6f7d7385b0816b88ae7519369256ceae2431f59473ab7a0917902b590024730440220241ae27c027f880f10db173447e582893de75aca5bfca9d547d95a592c9a53ec0220456e9291b0cf4c3e31b4a20bd14ae25cd8b175e5c6cf293db117887551e988370121033d954c5326559213b14e37cf029db0f8a75bb7bbac4644ce19a4834b1c71dee900000000


#### Broadcast child tx

In [21]:
merch_child_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + merch_child_tx + " true")
print(merch_child_txid)

defc3281489701d40517c8a8119eb07e874dc5145edceeb0380a64a9cb667136


In [22]:
mined_merch_child_txid = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_child_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + mined_merch_child_txid)
print(decoded)

{
  "txid": "defc3281489701d40517c8a8119eb07e874dc5145edceeb0380a64a9cb667136",
  "hash": "c774120b46f3b9cf87b1be082434893f558e64dbabc1f77055a9169b0a4f6410",
  "version": 2,
  "size": 339,
  "vsize": 177,
  "weight": 708,
  "locktime": 0,
  "vin": [
    {
      "txid": "9cef5178b6464d566532aad2510608a6c4c715490c7751b8d91387ba909905e6",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402203902507c229075f32eb6f9e2cc1c586219fcf993c12bc77724bf0492fd563043022024ff8a53a4d999f64793584aba73332db462b95e5eb35d7489dcd369129fa91201",
        "025bdf02f6aeb3a5cbfdece604956a49571d823b8888f5c4d3ceb58e453c044e57"
      ],
      "sequence": 4294967295
    },
    {
      "txid": "514c7ef5d1d94d6774753d78296a2b62b1b7f6e61cddbde6918bf7e243b536d9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30440220335e18eff0525006a03e78be8657e3addf6e2cb0b7b53f98ed4db32a30dd5de9022